# 0. Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Preamble analysis of the dataset

In [ ]:
df = pd.read_csv("./data/covid_data.csv");
df.head()

In [ ]:
boolean_columns = ["INTUBED", "PNEUMONIA", "PREGNANT", "DIABETES", "COPD", "ASTHMA", "INMSUPR", "HIPERTENSION", "OTHER_DISEASE", "CARDIOVASCULAR", "OBESITY", "RENAL_CHRONIC", "TOBACCO", "ICU"]

# Replace 97 and 99 with NA for the boolean columns
df[boolean_columns] = df[boolean_columns].replace({1: True, 2: False, 97: None, 99: None})
df.head()

In [ ]:
sns.heatmap(df.isnull(), cbar=False)
plt.show()